<a href="https://colab.research.google.com/github/ellesummerfield/programming-assignment-0/blob/main/Copy_of_Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [11]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [12]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())

In [ ]:
#Describe your answer below.
#Answer 1: First, the code gets a text file of data from the internet at a certain link and organizes 
# that data by separating it into columns based on the | symbol in the data. It then recreates a cleaner 
# version of the data, which only has rows for a company without a test issue. Next, the code creates
# a set of all the NASDAQ symbols from companies with no test issues. The second part takes similar data
# but with the ACT symbols for these companies and creates a set of all of the NASDAQ symbols from that 
# data from only again companies with no test issues. 

In [15]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2022-01-26 20:52:49--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2022-01-26 20:52:50--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1b73614594f23fe533c800edd6.dl.dropboxusercontent.com/cd/0/get/BeiQCpc0uUhbYuuOYNyOoccj0MOawKeZvUxmVcsFvzA44IP7Ou96KCqj9Sd8ton6CNQcdYz7C63NcDm1SiVCseI99ZTzDhZZk2pCLiAPDRTveydNekQ0QxyKulGdN0qfzIdNxNKa-fdiYNUoQTuSOuG-/file?dl=1# [following]
--2022-01-26 20:52:51--  https://uc1b73614594f23fe533c800edd6.dl.dropboxusercontent.com/cd/0/get/BeiQCpc0uUhbYuuOYNyOoccj0MOawKeZvUxmVcsFvzA44IP7Ou96KCqj9Sd8ton6CNQcdYz7C63NcDm

In [16]:
#!ls stock_closing_prices.parquet -l #downloaded file
!ls stocks.parquet -l

-rw-r--r-- 1 root root 678640589 Jan 26 20:54 stocks.parquet


In [5]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list.
!wget -O symbols_data http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt


--2022-01-26 17:44:58--  http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt
Resolving www.nasdaqtrader.com (www.nasdaqtrader.com)... 198.55.199.49
Connecting to www.nasdaqtrader.com (www.nasdaqtrader.com)|198.55.199.49|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 959616 (937K) [text/plain]
Saving to: ‘symbols_data’

symbols_data        100%[===================>] 937.12K   205KB/s    in 4.8s    

2022-01-26 17:45:03 (196 KB/s) - ‘symbols_data’ saved [959616/959616]



In [17]:
data=pd.read_parquet('stocks.parquet')

In [18]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock



,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [19]:
data

,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday
1,1962-01-03,0.124177,0.124177,0.121516,0.122846,0.044793,507341.0,0.0,0.0,HPQ,3,1962,1,Wednesday
2,1962-01-04,0.122846,0.126838,0.117968,0.120185,0.043823,845568.0,0.0,0.0,HPQ,4,1962,1,Thursday
3,1962-01-05,0.119742,0.119742,0.117525,0.117525,0.042853,338227.0,0.0,0.0,HPQ,5,1962,1,Friday
4,1962-01-08,0.117525,0.119299,0.115307,0.119299,0.043500,873754.0,0.0,0.0,HPQ,8,1962,1,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30305073,2021-12-20,0.510100,0.500000,0.500000,0.500000,0.500000,1.0,0.0,0,XPAXW,20,2021,12,Monday
30305074,2021-12-21,0.000000,0.750000,0.750000,0.750000,0.750000,2500.0,0.0,0,CDAQW,21,2021,12,Tuesday
30305075,2017-06-16,0.000000,0.000000,0.000000,25.250000,25.250000,0.0,0.0,0,MH-D,16,2017,6,Friday
30305076,2021-12-22,0.000000,0.500000,0.500000,0.500000,0.500000,400.0,0.0,0,JAQCW,22,2021,12,Wednesday


In [35]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

# data.sort_values(by="date")
# data.groupby(by="symbol")

smdata = data[['date', 'adj_close', 'symbol']]
smdata['date']= pd.to_datetime(smdata['date'])

dates = smdata.groupby('symbol', as_index=False).agg(first_date=pd.NamedAgg(column='date', aggfunc="min"), last_date=pd.NamedAgg(column='date', aggfunc="max"))
dates.head(2)
 
diff = dates.merge(smdata.rename(columns={'adj_close':'first_adj_close'}), left_on=['symbol','first_date'],right_on=['symbol','date']).drop('date', axis=1)
diff = diff.merge(smdata.rename(columns={'adj_close':'last_adj_close'}), left_on=['symbol', 'last_date'], right_on=['symbol','date']).drop('date', axis=1)
diff['days']= (diff.last_date - diff.first_date).dt.days
diff.head(3)

diff['return']=diff.apply(lambda row: None if row.first_adj_close == 0 or row.days == 0 else row.last_adj_close/row.first_adj_close/row.days, axis=1)

diff = diff.sort_values('return', ascending=False)
diff = diff.head(20)
diff



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,symbol,first_date,last_date,first_adj_close,last_adj_close,days,return
7126,NVO,1981-04-30,2021-12-27,2.275794e-09,111.629997,14851,3.302876e+06
6925,NKE,1980-12-02,2021-12-27,3.775067e-06,167.580002,15000,2.959418e+03
2420,DCI,1980-03-17,2021-12-27,9.812100e-06,57.939999,15260,3.869564e+02
9747,TJX,1987-06-26,2021-12-27,2.857077e-05,74.290001,12603,2.063167e+02
10765,WTRG,1980-03-17,2021-12-27,4.768529e-04,52.549999,15260,7.221605e+00
9932,TTC,1980-03-17,2021-12-27,1.051490e-03,99.879997,15260,6.224708e+00
10164,USB,1973-05-03,2021-12-27,7.828808e-04,56.740002,17770,4.078555e+00
6292,MCD,1966-07-05,2021-12-27,3.923203e-03,268.239990,20264,3.374097e+00
7729,PII,1987-09-16,2021-12-27,3.345692e-03,107.000000,12521,2.554223e+00
4565,HD,1981-09-22,2021-12-27,1.823626e-02,404.089996,14706,1.506773e+00


In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS

# Colab with AWS took slightly longer than the 40 seconds on Colab